In [1]:
ROOT_DIR = '/gpfs/commons/groups/gursoy_lab/aelhussein/ot_cost/otcost_fl'
import pandas as pd
import sys
import numpy as np
import pickle

In [2]:
def load_results(DATASET, reg_param = False):
    if reg_param:
        with open(f'{ROOT_DIR}/results/reg_param_tuning/{DATASET}_reg_param_tuning.pkl', 'rb') as f :
            results = pickle.load(f)
    else:
        with open(f'{ROOT_DIR}/results/lr_tuning/{DATASET}_lr_tuning.pkl', 'rb') as f :
            results = pickle.load(f)
    return results

# LR only

### Synthetic

In [4]:
DATASET = 'Synthetic'
lr_tuning = load_results(DATASET, reg_param = False)

In [14]:
len(lr_tuning[0.1]['single'][0.0005]['global']['losses'])

1

In [15]:
def get_structure(d, indent=0):
    structure = {}
    for key, value in d.items():
        if isinstance(value, dict):
            structure[key] = get_structure(value)
        elif isinstance(value, list) and value and isinstance(value[0], dict):
            structure[key] = [get_structure(value[0])]
        else:
            structure[key] = type(value).__name__
    return structure

# Example usage
from pprint import pprint
structure = get_structure(lr_tuning)
pprint(structure)

{0.1: {'ditto': {0.0005: {'global': {'losses': 'list', 'scores': 'list'},
                          'sites': {'client_1': {'losses': 'list',
                                                 'scores': 'list'},
                                    'client_2': {'losses': 'list',
                                                 'scores': 'list'}}}},
       'fedavg': {0.0005: {'global': {'losses': 'list', 'scores': 'list'},
                           'sites': {'client_1': {'losses': 'list',
                                                  'scores': 'list'},
                                     'client_2': {'losses': 'list',
                                                  'scores': 'list'}}}},
       'joint': {0.0005: {'global': {'losses': 'list', 'scores': 'list'},
                          'sites': {'client_joint': {'losses': 'list',
                                                     'scores': 'list'}}}},
       'pfedme': {0.0005: {'global': {'losses': 'list', 'scores': 'list'},
      

In [17]:
def analyze_results(data):
    """
    Analyzes federated learning results to find best LR rates and scores for each cost-server combination.
    
    Args:
        data: Nested dictionary containing the results
        
    Returns:
        Dictionary with analysis results for each cost-server combination
    """
    analysis_results = {}
    
    # Iterate through each cost level
    for cost in data.keys():
        analysis_results[cost] = {}
        
        # Iterate through each server type
        for server_type in data[cost].keys():
            min_loss = float('inf')
            best_lr = None
            best_score = float('-inf')
            
            # Iterate through learning rates
            for lr in data[cost][server_type].keys():
                # Get global losses for this configuration
                global_losses = data[cost][server_type][lr]['global']['losses']
                global_scores = data[cost][server_type][lr]['global']['scores']
                
                # Calculate average of minimum losses from each run
                if isinstance(global_losses, list):
                    avg_min_loss = sum(min(run) if isinstance(run, list) else float('inf') 
                                     for run in global_losses) / len(global_losses)
                    
                    # Update if this is the best loss we've seen for this cost-server combo
                    if avg_min_loss < min_loss:
                        min_loss = avg_min_loss
                        best_lr = lr
                
                # Track best score
                if isinstance(global_scores, list):
                    max_score = max(max(run) if isinstance(run, list) else float('-inf') 
                                  for run in global_scores)
                    best_score = max(best_score, max_score)
            
            analysis_results[cost][server_type] = {
                'best_lr': best_lr,
                'min_loss': min_loss,
                'best_score': best_score
            }
    
    return analysis_results

# Example usage:
results = analyze_results(lr_tuning)

# Print results in a formatted way
for cost in sorted(results.keys()):
    print(f"\nCost: {cost}")
    for server_type in sorted(results[cost].keys()):
        res = results[cost][server_type]
        print(f"  {server_type}:")
        print(f"    Best LR: {res['best_lr']}")
        print(f"    Min Loss: {res['min_loss']:.4f}")
        print(f"    Best Score: {res['best_score']:.4f}")


Cost: 0.1
  ditto:
    Best LR: 0.0005
    Min Loss: 1.4320
    Best Score: 0.7195
  fedavg:
    Best LR: 0.0005
    Min Loss: 0.7146
    Best Score: 0.7208
  joint:
    Best LR: 0.0005
    Min Loss: 1.0868
    Best Score: 0.6196
  pfedme:
    Best LR: 0.0005
    Min Loss: 0.8717
    Best Score: 0.7519
  single:
    Best LR: 0.0005
    Min Loss: 0.7199
    Best Score: 0.7554

Cost: 0.2
  ditto:
    Best LR: 0.0005
    Min Loss: 1.0990
    Best Score: 0.1396
  fedavg:
    Best LR: 0.0005
    Min Loss: 0.7972
    Best Score: 0.7843
  joint:
    Best LR: 0.0005
    Min Loss: 1.8610
    Best Score: 0.7059
  pfedme:
    Best LR: 0.0005
    Min Loss: 1.7726
    Best Score: 0.7478
  single:
    Best LR: 0.0005
    Min Loss: 0.9681
    Best Score: 0.7369

Cost: 0.3
  ditto:
    Best LR: 0.0005
    Min Loss: 0.7982
    Best Score: 0.6172
  fedavg:
    Best LR: 0.0005
    Min Loss: 0.7940
    Best Score: 0.6361
  joint:
    Best LR: 0.0005
    Min Loss: 0.8290
    Best Score: 0.7538
  pfedme:
 

In [22]:

# Process the dictionary
results = {}

for cost, server_types in lr_tuning.items():
    results[cost] = {}
    for server_type, lr_rates in server_types.items():
        best_lr = None
        best_loss_avg = float("inf")
        best_score = float("-inf")
        
        for lr, values in lr_rates.items():
            # Calculate average of the minimum global losses
            global_losses = values["global"]["losses"]
            min_losses = [min(run) for run in global_losses]  # Minimum loss for each run
            avg_min_loss = sum(min_losses) / len(min_losses)  # Average of the minimums

            # Update the best LR for average minimum loss
            if avg_min_loss < best_loss_avg:
                best_loss_avg = avg_min_loss
                best_lr = lr
            
        # Update the best score
        best_score = [max(m) for m in lr_rates[lr]["global"]["scores"]]
        # Store results for this server type
        results[cost][server_type] = {
            "best_lr": best_lr,
            "best_avg_min_loss": best_loss_avg,
            "best_score": best_score,
        }

# Print results
import pprint
pprint.pprint(results)

{0.1: {'ditto': {'best_avg_min_loss': 1.4320052862167358,
                 'best_lr': 0.0005,
                 'best_score': [0.7194877951180472]},
       'fedavg': {'best_avg_min_loss': 0.7146069705486298,
                  'best_lr': 0.0005,
                  'best_score': [0.7207670134102018]},
       'joint': {'best_avg_min_loss': 1.0867972373962402,
                 'best_lr': 0.0005,
                 'best_score': [0.6195652173913043]},
       'pfedme': {'best_avg_min_loss': 0.8717212975025177,
                  'best_lr': 0.0005,
                  'best_score': [0.7518593644354293]},
       'single': {'best_avg_min_loss': 0.7199075520038605,
                  'best_lr': 0.0005,
                  'best_score': [0.7554347826086957]}},
 0.2: {'ditto': {'best_avg_min_loss': 1.0989928841590881,
                 'best_lr': 0.0005,
                 'best_score': [0.13958810068649885]},
       'fedavg': {'best_avg_min_loss': 0.797239363193512,
                  'best_lr': 0.0005,
      

In [3]:
import torch

In [4]:
efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)

Using cache found in /gpfs/commons/home/aelhussein/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/gpfs/commons/home/aelhussein/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/gpfs/commons/home/aelhussein/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


In [5]:
for name, module in efficientnet.named_modules():
    print(f"Layer name: {name}")
    print(module)
    print("----------")


Layer name: 
EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
   